In [1]:
import pandas as pd
from scipy.stats.stats import pearsonr
from scipy import stats
from scipy import special
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
#dfs = dfs by year (14, 15, 16, 17 separated)
def normalize_empirical_data(dfs, states):
    # cbar = sum over years, states, and insurers costs / sum enrollees
    cbar_numerator = 0
    cbar_denominator = 0
    for current_df in dfs:
        current_df = current_df.where(current_df["STATE"].isin(states)).dropna()
        print(len(current_df))
        cbar_numerator = cbar_numerator + sum(current_df["TotalCosts"])
        cbar_denominator = cbar_denominator + sum(current_df["Enrollees"])
    cbar = cbar_numerator / cbar_denominator
    
    new_dfs = []
    # c_ys = average cost in this year and state
    int_to_print = 14
    for current_df in dfs:
        new_df = state(current_df, states[0])
        c_ys = sum(new_df["TotalCosts"]) / sum(new_df["Enrollees"])
        new_df["TransfersNormalized"] = new_df["Transfers"] * cbar / c_ys
        print("%s %d Ratio is %.3f" % (states[0], int_to_print, cbar / c_ys))
        if len(states) > 1:
            for current_state in states[1:]:
                df3 = state(current_df, current_state)
                if sum(df3["Enrollees"]) > 0:
                    c_ys = sum(df3["TotalCosts"]) / sum(df3["Enrollees"])
                    df3["TransfersNormalized"] = df3["Transfers"] * cbar / c_ys
                    new_df = new_df.append(df3)
                    print("%s %d Ratio is %.3f" % (current_state, int_to_print, cbar / c_ys))
        int_to_print += 1
            
        new_dfs.append(new_df)
    
    empirical_transfers_sum = 0
    for df in new_dfs:
        empirical_transfers_sum = empirical_transfers_sum + sum(abs(df["TransfersNormalized"]))
    print("Empirical Transfers Sum = %.3f" % (empirical_transfers_sum))
    return new_dfs

In [3]:
def plot(df, label, show_graphs):
    no_nulls = df[['LogEnrollees', 'LogTransfers']].dropna()
    X = no_nulls[['LogEnrollees']]
    y = no_nulls['LogTransfers']
    X = sm.add_constant(X)
    est = sm.OLS(y, X).fit()
    equation = "Log T = %f + %f * Log Enrollees" % (est.params[0], est.params[1])
    if show_graphs:
        fig, ax = plt.subplots(figsize=(8,8))
        plt.scatter(df["LogEnrollees"], df["LogTransfers"])
        x = np.linspace(min(df["LogEnrollees"]), max(df["LogEnrollees"]), 1000)
        plt.plot(x, est.params[0] + est.params[1] * x, label=equation)
        plt.title(label)
        plt.xlabel("Log # Enrollees")
        plt.ylabel("Log Transfers^2 + 1")
        plt.legend()
        plt.show()
    print(label)
    print(equation)
    
def state(df, state):
    return df.where(df["STATE"] == state).where(df["LogTransfers"] > 2).dropna()

def remove_outliers(df):
    return df.where(df["LogTransfers"] > 2).dropna()

In [4]:
def erfinv2(sample_size):# sig):
    # erf inverse 0.5 * p/k where p=0.05
    print("Sample size %d" % (sample_size))
    return special.erfinv(1-0.5*0.05/sample_size) * np.sqrt(2)# * sig

def num_pass(df, a):
    sample_size = df.shape[0]
    count = 0
    delta = erfinv2(sample_size)
    print("Delta_0 % .3f" % (delta))
    es1 = max(abs(df["TransfersPerSqrtEnrollee"])) / delta
    print("Comparison es_0 = %.3f" % (es1))
    print("Comparison es_0*%.2f = %.3f" % (a, a*es1))
    
    for index, row in df.iterrows():
        t = row["Transfers"]
        n = row["Enrollees"]
        if abs(row["TransfersPerSqrtEnrollee"]) < a*es1:
            count += 1
            print("T/sqrt(N)=%.3f \t <a*es_0 Satisfied" % (row["TransfersPerSqrtEnrollee"]))
        else:
            print("T/sqrt(N)=%.3f \t <a*es_0 NOT Satisfied \t Distance |T/sqrt(N)|-a*es_0 %.1f " % (row["TransfersPerSqrtEnrollee"], abs(row["TransfersPerSqrtEnrollee"]) - a*es1))
    return count
    
def binom(k, n, p):
    return stats.binom.cdf(k, n, p)

In [5]:
def run_stat_test(data, label, a):
    print(label)
    p01 = special.erf(a/np.sqrt(2))
    print("P01 = %.3f" % (p01))
    successes = num_pass(data, a)
    print("Number of successes: %d" % (successes))
    print("P value = %.3f" % (1- binom(successes, data.shape[0], p01)))
    
def run_binom_stat_test2(beta, values):
    print("Beta (=std): %d" % (beta))
    above_2_count = 0
    for value in values:
        if abs(value) > 2*beta:
            above_2_count += 1
        #    print("value %d passed" % (value))
        #else:
        #    print("value %d failed" % (value))
    p_above_2 = 1 - special.erf(np.sqrt(2))
    print("Number of T/sqrt(n) above 2 stds: %d" % (above_2_count))
    print("Proportion of T/sqrt(n) above 2 stds: %.3f" % (above_2_count / len(values)))
    print("P value = %.6f" % (1- binom(above_2_count, len(values), p_above_2)))

In [6]:
def ratio(df, v):# sig):
    # # of people where |d_i| < v
    return df["TransfersNormalizedPerSqrtEnrollee"].where(abs(df["TransfersNormalizedPerSqrtEnrollee"]) < v).dropna().count() / df["TransfersNormalizedPerSqrtEnrollee"].count()

# the ratio function, just on lists instead of df's
def ratio_list(t, v):# sig):
    # # of people where |d_i| < v
    k = len(t)
    t2 = [x for x in t if abs(x) < v]
    return len(t2) / k

In [7]:
def create_simulation(states, dfs, es):
    all_transfers3 = list()
    mu = 0

    for a in range(15):
        for current_state in states:
            for current_df in dfs:
                df3 = state(current_df, current_state)
                if len(df3) > 0:
                    all_costs = list()
                    for _, row in df3.iterrows():
                        all_costs.append(np.random.normal(mu, es* np.sqrt(row["Enrollees"])))

                    # normalize by state/year
                    # calculate average costs for a given year/state, normalize by it
                    c_mean = sum(all_costs) / sum(df3["Enrollees"])

                    i = 0
                    for _, row in df3.iterrows():
                        all_transfers3.append((all_costs[i] - c_mean*row["Enrollees"]) / np.sqrt(row["Enrollees"]))  
                        i += 1
            
    ratios_c3 = [ratio_list(all_transfers3, vi) for vi in v]
    return ratios_c3

In [8]:
def create_simulation_adjust_for_imbalances(states, dfs, es):
    all_transfers3 = list()
    mu = 0

    for z in range(15):
        sum_sqrt_a = 0
        for current_state in states:
            for current_df in dfs:
                df3 = state(current_df, current_state)
                if len(df3) > 0:
                    all_costs = list()
                    sum_n = sum(df3["Enrollees"])
                    for _, row in df3.iterrows():
                        n_i = row["Enrollees"] 
                        sum_n_minus_n_i = sum_n - n_i
                        a = n_i * (sum_n_minus_n_i / sum_n)**2 + sum_n_minus_n_i * (n_i / sum_n)**2
                        #print("n_i= %.3f; a= %.3f" % (n_i, a))
                        sqrt_a = np.sqrt(a)
                        sum_sqrt_a = sum_sqrt_a + sqrt_a
                        all_costs.append(np.random.normal(mu, es* sqrt_a))

                    # normalize by state/year
                    # calculate average costs for a given year/state, normalize by it
                    c_mean = sum(all_costs) / sum(df3["Enrollees"])

                    i = 0
                    for _, row in df3.iterrows():
                        all_transfers3.append((all_costs[i] - c_mean*row["Enrollees"]) / np.sqrt(row["Enrollees"]))  
                        i += 1
            
    ratios_c3 = [ratio_list(all_transfers3, vi) for vi in v]
    
    print("Sum sqrt(a) = %.3f" % (sum_sqrt_a))
    return ratios_c3, sum_sqrt_a

In [9]:
def run_test_against_theoretical(df2, starting_beta, ratios2):
    print("k = %d" % (len(df2)))

    greatest_diff = 1
    test_stat = special.kolmogi(0.05)/np.sqrt(k)
    print("test_stat at this k = %.5f" % (test_stat))

    beta1 = starting_beta

    while greatest_diff > test_stat:
        theoretical = [special.erf(vi/(beta1*np.sqrt(2))) for vi in v]

        Mb_index = -1
        greatest_diff = 0
        for i, vi in enumerate(v):
            vb =  ratios2[i]-theoretical[i]
            if vb > greatest_diff:
                greatest_diff = vb
                Mb_index = i

        if greatest_diff > test_stat:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (theoretical[Mb_index]))
            print("Difference = %.4f" % (greatest_diff))
            #print("beta at this point = %.3f" % (beta[Mb_index]))
            #print(greatest_diff > test_stat)

            print("\n\n")
            beta1 = beta1 - 10000

    beta1 = beta1 + 10000
    greatest_diff = 1

    while greatest_diff > test_stat:
        theoretical = [special.erf(vi/(beta1*np.sqrt(2))) for vi in v]

        Mb_index = -1
        greatest_diff = 0
        for i, vi in enumerate(v):
            vb =  ratios2[i]-theoretical[i]
            #print(vb)
            if vb > greatest_diff:
                greatest_diff = vb
                Mb_index = i

        if greatest_diff > test_stat:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (theoretical[Mb_index]))
            print("Difference = %.4f" % (greatest_diff))
            #print("beta at this point = %.3f" % (beta[Mb_index]))
            #print(greatest_diff > test_stat)

            print("\n\n")
            beta1 = beta1 - 100
    return beta1, Mb_index

In [10]:
def run_test_against_simulation(states, dfs, starting_beta, ratios2):
    beta1 = starting_beta
    print("k = %d" % (len(df2)))
    greatest_diff = 1 
    test_stat = special.kolmogi(0.05)/np.sqrt(k)
    print("test_stat at this k = %.5f" % (test_stat))

    while greatest_diff > test_stat:
        simulation = create_simulation(states, dfs, beta1)

        Mb_index = -1
        greatest_diff = 0
        for i, vi in enumerate(v):
            vb =  ratios2[i]-simulation[i]
            if vb > greatest_diff:
                greatest_diff = vb
                Mb_index = i

        if greatest_diff > test_stat:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f" % (greatest_diff))
            #print("beta at this point = %.3f" % (beta[Mb_index]))
            #print(greatest_diff > test_stat)

            print("\n\n")
            beta1 = beta1 - 10000
        else:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f"  % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f NOT > test_stat" % (greatest_diff))

    beta1 = beta1 + 10000
    greatest_diff = 1

    while greatest_diff > test_stat:
        simulation = create_simulation(states, dfs, beta1)
        Mb_index = -1
        greatest_diff = 0
        for i, vi in enumerate(v):
            vb =  ratios2[i]-simulation[i]
            if vb > greatest_diff:
                greatest_diff = vb
                Mb_index = i

        if greatest_diff > test_stat:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f" % (greatest_diff))
            #print("beta at this point = %.3f" % (beta[Mb_index]))
            #print(greatest_diff > test_stat)
            print("\n\n")
            beta1 = beta1 - 100
        else:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f NOT > test_stat" % (greatest_diff))
    return beta1, Mb_index

In [11]:
def run_test_against_simulation_adjust_for_imbalances(states, dfs, starting_beta, ratios2):
    beta1 = starting_beta
    print("k = %d" % (len(df2)))
    greatest_diff = 1
    test_stat = special.kolmogi(0.05)/np.sqrt(k)
    print("test_stat at this k = %.5f" % (test_stat))

    while greatest_diff > test_stat:
        simulation, _ = create_simulation_adjust_for_imbalances(states, dfs, beta1)

        Mb_index = -1
        greatest_diff = 0
        for i, vi in enumerate(v):
            vb =  ratios2[i]-simulation[i]
            if vb > greatest_diff:
                greatest_diff = vb
                Mb_index = i

        if greatest_diff > test_stat:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f" % (greatest_diff))
            #print("beta at this point = %.3f" % (beta[Mb_index]))
            #print(greatest_diff > test_stat)

            print("\n\n")
            beta1 = beta1 - 10000
        else:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f NOT > test_stat" % (greatest_diff))

    beta1 = beta1 + 10000
    greatest_diff = 1

    while greatest_diff > test_stat:
        simulation, sum_sqrt_a = create_simulation_adjust_for_imbalances(states, dfs, beta1)
        Mb_index = -1
        greatest_diff = 0
        for i, vi in enumerate(v):
            vb =  ratios2[i]-simulation[i]
            if vb > greatest_diff:
                greatest_diff = vb
                Mb_index = i

        if greatest_diff > test_stat:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f" % (greatest_diff))
            #print("beta at this point = %.3f" % (beta[Mb_index]))
            #print(greatest_diff > test_stat)
            print("\n\n")
            beta1 = beta1 - 100
        else:
            print("\nAt the point of maximum difference with beta = %d:" % (beta1))
            print("P(|d_i|<v) = %.4f" % (ratios2[Mb_index]))
            print("P(|N(0,max_beta^2)|<v) = %.4f" % (simulation[Mb_index]))
            print("Difference = %.4f NOT > test_stat" % (greatest_diff))
            print("E(|X|) = %.3f" % (np.sqrt(2)*beta1*sum_sqrt_a/np.pi))
    return beta1, Mb_index

In [12]:
# df2 is a combination of states' values of transfers per sqrt(enrollee) over years

def create_df2(states, dfs, label):
    current_state = states[0]
    
    df2 = state(dfs[0], current_state)[label]

    for df in dfs[1:]:
        df2 = df2.append(state(df, current_state)[label]) 

    if len(states) > 1:
        for current_state in states[1:]:
            for df in dfs:
                df2 = df2.append(state(df, current_state)[label]) 

    return df2

In [13]:
# df3 is a combination of states' values over years but includes # enrollees
# in addition to transfers per sqrt(enrollee)

def create_df3(states, dfs):
    current_state = states[0]
    
    df3 = state(dfs[0], current_state)

    for df in dfs[1:]:
        df3 = df3.append(state(df, current_state), sort=False) 

    if len(states) > 1:
        for current_state in states[1:]:
            for df in dfs:
                df3 = df3.append(state(df, current_state), sort=False) 

    return df3